# Wind Rose

### Packages

In [16]:
import xarray as xr
import numpy as np
import pandas as pd

import sys
import os
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath("d:/Users/julia/Desktop/Academia/01_Mestrado/Work/Utils"))
from HistDir import *

### Metar Database

#### Importing Data

In [17]:
Metar = pd.DataFrame({'label':['SBNT','SBJP','SBRF','SBMO','SBAR','SBSV'],
                      'Latitude':[-5.91,-7.15,-8.13,-9.51,-10.98,-12.91],
                      'Longitude': [-35.25,-34.97,-34.92,-35.79,-37.07,-38.33]})

i = [0,1,2,3,4,5]
Names = ['SBNT','SBJP','SBRF','SBMO','SBAR','SBSV']

df_list =[]
for n in range(len(Names)):
    df = pd.read_csv("D:/Users/julia/Desktop/AcaDemia/01_MestraDo/Dados/METAR/"+ Names[n] +".csv",sep=',',
    index_col='times', parse_dates=True)
    df_list.append(df)

#### Hist_Dir

##### Total

In [18]:
for i,n in enumerate(Names):
    dft = df_list[i] # Getting one station from the dataframe
    print(n)

    vel = dft.speed
    dire = dft.direction
    outputname = 'd:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/METAR/Figuras/His_Dir/Serie_Total/Norte/'+n

    HistDir(P=vel,D=dire,Pmax=12,MaxProb=50,
            arqname=outputname,
            par='vento',interpolado=False,conv_oc=True)

SBNT
Done
SBJP
Done
SBRF
Done
SBMO
Done
SBAR
Done
SBSV
Done


##### Sazonal - DJF MAM JJA SON

In [19]:
for i,n in enumerate(Names):
    dft = df_list[i] # Getting one station from the dataframe
    print(n)
    
    ##### DIVISÃO PARA ESTUDOS SAZONAIS #####

    season = ((dft.index.month % 12 + 3) // 3).map({1:'DJF', 2: 'MAM', 3:'JJA', 4:'SON'})
    
    for i,s in enumerate(season.unique()):
        # print(s)
        vel_s = dft[season == s].speed
        dire_s = dft[season == s].direction

        outputname_s = 'd:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/METAR/Figuras/His_Dir/Serie_Total/Norte/Metar_S1/'+n+'_'+s

        HistDir(P=vel_s,D=dire_s,Pmax=12,MaxProb=60,
                arqname=outputname_s,
                par='vento',interpolado=False,conv_oc=True)

SBNT
Done
Done
Done
Done
SBJP
Done
Done
Done
Done
SBRF
Done
Done
Done
Done
SBMO
Done
Done
Done
Done
SBAR
Done
Done
Done
Done
SBSV
Done
Done
Done
Done


##### Sazonal - JFM AMJ JAS OND

In [20]:

for i,n in enumerate(Names):
    dft = df_list[-i] # Getting one station from the dataframe
    print(n)

    ##### DIVISÃO PARA ESTUDOS SAZONAIS #####
    dft_summer = dft[dft.index.month.isin([1,2,3])]
    dft_autumn = dft[dft.index.month.isin([4,5,6])]
    dft_winter = dft[dft.index.month.isin([7,8,9])]
    dft_spring = dft[dft.index.month.isin([10,11,12])]

    season_names = ['Sum','Aut','Win','Spr']

    for d,s in zip([dft_summer, dft_autumn, dft_winter, dft_spring],season_names):

        vel_s = d.speed
        dire_s = d.direction

        outputname_s = 'd:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/METAR/Figuras/His_Dir/Serie_Total/Norte/Metar_S2/'+n+'_'+s

        HistDir(P=vel_s,D=dire_s,Pmax=12,MaxProb=60,
                arqname=outputname_s,
                par='vento',interpolado=False,conv_oc=True)

SBNT
Done
Done
Done
Done
SBJP
Done
Done
Done
Done
SBRF
Done
Done
Done
Done
SBMO
Done
Done
Done
Done
SBAR
Done
Done
Done
Done
SBSV
Done
Done
Done
Done


### ERA5 Database

#### Import and Setting the Data

In [21]:
dsmerged = xr.open_dataset('D:/Users/julia//Desktop/Academia/01_Mestrado/Dados/ERA5.nc',)

In [22]:
era_SBNT = dsmerged.sel(latitude=-5.8, longitude=-35.1, method='nearest') # 2º point-5.8 -34.85
u_SBNT = era_SBNT['u10']
v_SBNT = era_SBNT['v10']

era_SBJP = dsmerged.sel(latitude=-7.05, longitude=-34.6, method='nearest') # 2º point-7.05 -34.35
u_SBJP = era_SBJP['u10']
v_SBJP = era_SBJP['v10']

era_SBRF = dsmerged.sel(latitude=-8.05, longitude=-34.6, method='nearest') # 2º point-8.05 -34.35
u_SBRF = era_SBRF['u10']
v_SBRF = era_SBRF['v10']

era_SBMO = dsmerged.sel(latitude=-9.55, longitude=-35.35, method='nearest') # 2º point-9.55 -35.1
u_SBMO = era_SBMO['u10']
v_SBMO = era_SBMO['v10']

era_SBAR = dsmerged.sel(latitude=-11.05, longitude=-36.85, method='nearest') # 2º point -11.05 -36.6
u_SBAR = era_SBAR['u10']
v_SBAR = era_SBAR['v10']

era_SBSV = dsmerged.sel(latitude=-13.05, longitude=-38.35, method='nearest') # 2º point -12.8 -37.85/-38.1
u_SBSV = era_SBSV['u10']
v_SBSV = era_SBSV['v10']

In [23]:
era = [era_SBNT,era_SBJP,era_SBRF,era_SBMO,era_SBAR,era_SBSV]
names = ['SBNT','SBJP','SBRF','SBMO','SBAR','SBSV']     

#### Hist_Dir

##### Total

In [24]:
for e,n in zip(era,names):
        print(n)
        
        e = e.assign(vel=lambda e: np.sqrt(e['u10']**2+e['v10']**2))
        e = e.assign(dire=lambda e: (np.mod(180+np.rad2deg(np.arctan2(e['u10'], e['v10'])),360)))
        # e.dire[e.dire<0] += 360

        outputname = 'd:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/ERA/'+n+''

        HistDir(P=e.vel,D=e.dire,Pmax=12,MaxProb=50,
                arqname=outputname,
                par='vento',interpolado=False,conv_oc=True)

SBNT
Done
SBJP
Done
SBRF
Done
SBMO
Done
SBAR
Done
SBSV
Done


##### Sazonal - DJF MAM JJA SON

In [25]:
Season = [[12,1,2],[3,4,5],[6,7,8],[9,10,11]]
Season_names = ['SUMMER','AUTUMN','WINTER','SPRING']

for e,n in zip(era,names):
    print(n)

    e = e.assign(vel=lambda e: np.sqrt(e['u10']**2+e['v10']**2))
    e = e.assign(dire=lambda e: (np.mod(180+np.rad2deg(np.arctan2(e['u10'], e['v10'])),360)))

    for s,sn in zip(Season,Season_names):

        vel_s =  e['vel'].sel(time=e.time.dt.month.isin(s))
        dire_s = e['dire'].sel(time=e.time.dt.month.isin(s))

        outputname_s = 'd:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/ERA/ERA_S1/'+n+'_'+sn

        HistDir(P=vel_s,D=dire_s,Pmax=12,MaxProb=60,
                arqname=outputname_s,
                par='vento',interpolado=False,conv_oc=True)

SBNT
Done
Done
Done
Done
SBJP
Done
Done
Done
Done
SBRF
Done
Done
Done
Done
SBMO
Done
Done
Done
Done
SBAR
Done
Done
Done
Done
SBSV
Done
Done
Done
Done


##### Sazonal - JFM AMJ JAS OND

In [26]:
Season = [[1,2,3],[4,5,6],[4,5,6],[10,11,12]]
Season_names = ['SUMMER','AUTUMN','WINTER','SPRING']

for e,n in zip(era,names):
    print(n)

    e = e.assign(vel=lambda e: np.sqrt(e['u10']**2+e['v10']**2))
    e = e.assign(dire=lambda e: (np.mod(180+np.rad2deg(np.arctan2(e['u10'], e['v10'])),360)))

    for s,sn in zip(Season,Season_names):

        vel_s =  e['vel'].sel(time=e.time.dt.month.isin(s))
        dire_s = e['dire'].sel(time=e.time.dt.month.isin(s))

        outputname_s = 'd:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/ERA/ERA_S2/'+n+'_'+sn

        HistDir(P=vel_s,D=dire_s,Pmax=12,MaxProb=60,
                arqname=outputname_s,
                par='vento',interpolado=False,conv_oc=True)

SBNT
Done
Done
Done
Done
SBJP
Done
Done
Done
Done
SBRF
Done
Done
Done
Done
SBMO
Done
Done
Done
Done
SBAR
Done
Done
Done
Done
SBSV
Done
Done
Done
Done
